<a href="https://colab.research.google.com/github/lkarjun/malayalam-language-model/blob/main/Malayalam-Language-Model/malayalam-language-model_awd_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Downloading

In [44]:
!pip install -qq dvc[gdrive]

!dvc get https://github.com/lkarjun/malayalam-language-model \
Datasets/

!unzip -q 'Datasets/*.zip' -d Datasets/


    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Enter verification code: 4/1AX4XfWjm6SqYTmDl62YT2Dmy5xjb-fMKRJQ45L9hAox_k9lJByv18_qN7OY
Authentication successful.

3 archives were successfully processed.


## Malayalam Language Model AWD-LSTM

### Imports

In [1]:
!pip install -qq fastai==2.5.3 sentencepiece transformers

     |████████████████████████████████| 189 kB 7.3 MB/s 
     |████████████████████████████████| 1.2 MB 43.9 MB/s 
     |████████████████████████████████| 3.5 MB 33.1 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 35.8 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
     |████████████████████████████████| 6.8 MB 45.9 MB/s 
     |████████████████████████████████| 67 kB 3.9 MB/s 


In [58]:
from fastai.text.all import *
from transformers import PreTrainedTokenizerFast

import pandas as pd
from tqdm import tqdm

DATASET = Path("/content/Datasets/")

tqdm.pandas(colour = 'red')

In [59]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("lkarjun/malayalam-language-model", 
                                                    use_auth_token=True,
                                                    )

### Dataset Loading

In [60]:
def open_text_files(f: str):
  return Path(f).read_text(encoding="utf-8")

In [61]:
files = ['article_files.csv', 'magazine_files.csv', 'wikitext_files.csv']
df = pd.concat([pd.read_csv(DATASET/f) for f in files], ignore_index = True)

In [62]:
df['content'] = df['file_path'].progress_apply(open_text_files)

100%|██████████| 35490/35490 [00:03<00:00, 11247.86it/s]


In [63]:
sample = df[:100]
len(sample)

100

### Tokenizer

In [64]:
class MalayalamTokenizer(Transform):
  def __init__(self, hf_tok: PreTrainedTokenizerFast):
    self.hf_tok = hf_tok
  
  def encodes(self, x): 
        toks = self.hf_tok.tokenize(x)
        toks = self.hf_tok.convert_tokens_to_ids(toks)
        return tensor(toks)

  def decodes(self, x): 
    x = self.hf_tok.decode(x.cpu().numpy())
    return TitledStr(x)


In [65]:
tok = MalayalamTokenizer(tokenizer)
tok

MalayalamTokenizer:
encodes: (object,object) -> encodes
decodes: (object,object) -> decodes

### Dataloder

In [67]:
splits = RandomSplitter(.1, 7)(df)

In [68]:
tfms = TfmdLists(df['content'], 
                 MalayalamTokenizer(tokenizer), 
                 splits=splits, 
                 dl_type=LMDataLoader)

In [70]:
mldls = tfms.dataloaders(bs=120, seq_len=72)

In [73]:
mldls.show_batch(max_n = 2)

,text,text_
0,തെങ്ങ് നനച്ച ##ാൽ വിളവ് ഇരട്ടി ##ക്കും. തുള്ളിനന ##യാണെങ്കിൽ ഒരു ദിവസം 60 – 70 ലീറ്റര്‍ നൽകുക. തടത്തിൽ വെള്ളം തുറന്നുവിട ##ുകയാണെങ്കിൽ ഒരു തവണ 400 – 700 ലീറ്റർ ##‌. ഇടവേള 5 – 8 ദിവസം. മണല ##ിന്റെ അംശം കൂടിയ മണ്ണിൽ ചെറിയ അളവും ചെറിയ ഇടവേള ##യും. കളിമണ്ണ ##ിന്റെ അംശം കൂടിയ മണ്ണിൽ കൂടുതൽ അളവും കൂടിയ ഇടവേള ##യും. മണല ##ിന്റെ അംശം കൂടിയ തടങ്ങളിൽ ചേ ##റും കളിമണ്ണ ##ിന്റെ അംശം കൂടിയ തടങ്ങളിൽ മണലും ചേർ ##‌,നനച്ച ##ാൽ വിളവ് ഇരട്ടി ##ക്കും. തുള്ളിനന ##യാണെങ്കിൽ ഒരു ദിവസം 60 – 70 ലീറ്റര്‍ നൽകുക. തടത്തിൽ വെള്ളം തുറന്നുവിട ##ുകയാണെങ്കിൽ ഒരു തവണ 400 – 700 ലീറ്റർ ##‌. ഇടവേള 5 – 8 ദിവസം. മണല ##ിന്റെ അംശം കൂടിയ മണ്ണിൽ ചെറിയ അളവും ചെറിയ ഇടവേള ##യും. കളിമണ്ണ ##ിന്റെ അംശം കൂടിയ മണ്ണിൽ കൂടുതൽ അളവും കൂടിയ ഇടവേള ##യും. മണല ##ിന്റെ അംശം കൂടിയ തടങ്ങളിൽ ചേ ##റും കളിമണ്ണ ##ിന്റെ അംശം കൂടിയ തടങ്ങളിൽ മണലും ചേർ ##‌ ##ക്കുന്നതു
1,"##ഡെ ##ടുത്ത ##ാലും അതിലൊക്കെ നിരവധി ആഴത്തിലുള്ള സൗഹൃദ ##ങ്ങളുള്ള ആ യുവാവിനെ മറ്റാർക്കും വിട്ടു ##നൽക ##ാതെ മുല ##ായം തന്റെ കൂടെത്തന്നെ നിലനിർത്തി. മുല ##ായ ##മാണ് അമർ സിങ്ങിനെ പ്രാഥമിക ##ാംഗ ##ത്വം നൽകി സമാജ് ##‌വ ##ാദി പാർട്ടിയിൽ എടുത്തത്. അന്ന് മുല ##ായം സിങിന്റെ നാലുപാടും, രാജ് ബബ ##്ബ ##ർ, ആസ ##ം ഖാൻ, റാം ഗോപാൽ യാദവ്, ബേ ##നി പ്രസാദ് വർമ്മ എന്നിങ്ങനെ നിരവധിപേർ അടുപ്പം സ്ഥാപിക്കാൻ മിനക്കെട ##്ട ##ു നടക്കുന്ന കാലം. അവരെയൊക്കെ നിരാശപ്പെടുത്ത ##ിക്കൊണ്ട് മുല","##ടുത്ത ##ാലും അതിലൊക്കെ നിരവധി ആഴത്തിലുള്ള സൗഹൃദ ##ങ്ങളുള്ള ആ യുവാവിനെ മറ്റാർക്കും വിട്ടു ##നൽക ##ാതെ മുല ##ായം തന്റെ കൂടെത്തന്നെ നിലനിർത്തി. മുല ##ായ ##മാണ് അമർ സിങ്ങിനെ പ്രാഥമിക ##ാംഗ ##ത്വം നൽകി സമാജ് ##‌വ ##ാദി പാർട്ടിയിൽ എടുത്തത്. അന്ന് മുല ##ായം സിങിന്റെ നാലുപാടും, രാജ് ബബ ##്ബ ##ർ, ആസ ##ം ഖാൻ, റാം ഗോപാൽ യാദവ്, ബേ ##നി പ്രസാദ് വർമ്മ എന്നിങ്ങനെ നിരവധിപേർ അടുപ്പം സ്ഥാപിക്കാൻ മിനക്കെട ##്ട ##ു നടക്കുന്ന കാലം. അവരെയൊക്കെ നിരാശപ്പെടുത്ത ##ിക്കൊണ്ട് മുല ##ായത്ത"


### Language Model

In [79]:
mldls.vocab = tokenizer.vocab

In [75]:
doc(language_model_learner)

language_model_learner(dls, arch, config=None, drop_mult=1.0, backwards=False, pretrained=True, pretrained_fnames=None, loss_func=None, opt_func=<function Adam at 0x7f0915e7e050>, lr=0.001, splitter=<function trainable_params at 0x7f0924349680>, cbs=None, metrics=None, path=None, model_dir='models', wd=None, wd_bn_bias=False, train_bn=True, moms=(0.95, 0.85, 0.95))
Create a `Learner` with a language model from `dls` and `arch`.

To get a prettier result with hyperlinks to source code and documentation, install nbdev: pip install nbdev


In [80]:
learn = language_model_learner(
            mldls, 
            AWD_LSTM,
            drop_mult=.3,
            pretrained = False,
            metrics = [accuracy, Perplexity()]
            ).to_fp16()

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
learn.fit(1, 3e-4)